<a href="https://colab.research.google.com/github/SourabhGothe/DA224-O-Mini-Projects/blob/main/Seoul_Rental_bike_count_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common Imports

In [1]:
from google.colab import drive
import os
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNet, SGDRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, GridSearchCV
import xgboost as xg
import numpy as np
import sys
from sklearn import linear_model
pd.options.mode.chained_assignment = None

from IPython.display import Markdown, display
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
np.random.seed(42)

In [2]:
drive.mount('/content/drive')
drivepath = "/content/drive/Shareddrives/DA224-O/mp_1/data/"

Mounted at /content/drive


## Load the dataset from Google Drive


In [101]:
data_file = drivepath+'SeoulBikeData_MP1.csv'
df = pd.read_csv (data_file, index_col=0)

# Common methods required

In [102]:
def prepareData(data, removeRedundant=True, max_days=None):
  X = data.drop(['Rented Bike Count'], axis=1).copy()
  y = data['Rented Bike Count'].copy()

  #If already prepared, just return the feature and target
  if 'Seasons_sin' in data.columns:
      if removeRedundant:
        X = X.drop(['Date'], axis=1)
        X = X.drop(['Hour'], axis=1)
        X = X.drop(['Seasons'], axis=1)
      return X, y
  
  X['Seasons'].replace(['Summer', 'Spring', "Autumn", "Winter"], [0, 1, 2, 3], inplace=True)
  X["Seasons_sin"] = np.sin(2 * np.pi * X['Seasons']/3.0)
  X['Date'] =  pd.to_datetime(X['Date'])
  X["Month"] = np.sin(2 * np.pi * X['Date'].dt.month/12.0)
  if max_days == None:
    max_days = 6.0
  X['Day'] =  np.sin(2 * np.pi * X['Date'].dt.dayofweek/max_days)
  X['Hour_sin'] = np.sin(2 * np.pi * X['Hour']/23.0)
  X['Hour_cos'] = np.cos(2 * np.pi * X['Hour']/23.0)

  if removeRedundant:
    X = X.drop(['Date'], axis=1)
    X = X.drop(['Hour'], axis=1)
    X = X.drop(['Seasons'], axis=1)
  return X, y

**Hyper-parameter tuning with Grid Search**

In [ ]:
model_params = {
    'elasticnet' : {
        'model' : ElasticNet(),
        'params' : {
            'alpha' : [0.1,0.15,0.18,0.2,0.22,0.24,0.3],
            'l1_ratio' : [0.9,0.95,0.97,0.98,0.99,1],
            'max_iter' : [20000,15000,25000] 
        }
    }
    ,
    'SGDRegressor' : {
        'model' : SGDRegressor(random_state = 42),
        'params' : {
            'alpha' : [0.1,0.01,0.001,0.0001],
            'max_iter' : [20000,15000,25000],
            'learning_rate' : ['constant','optimal','invscaling']
        }
    }
}

def runPipeline(X1):
  cat_cols = ['Holiday', 'Functioning Day']
  num_cols = [col for col in list(X1) if col not in cat_cols]

  num_pipeline = Pipeline([('imputer', KNNImputer(n_neighbors=2)), ('scaler',StandardScaler())])
  cat_pipeline = Pipeline([('encoder',OneHotEncoder())])
  full_pipeline = ColumnTransformer([('num', num_pipeline, num_cols),
                                  ('cat', cat_pipeline, cat_cols)])

  #Data preparation
  X_train_prepared = full_pipeline.fit_transform(X1)

  return X_train_prepared

def GridSCV(data):
  X_grid,y_grid = prepareData(data)
  X_prepared_grid = runPipeline(X_grid)
  scores = []
  for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'], cv=5, scoring ='neg_mean_squared_error', return_train_score = False)
    clf.fit(X_prepared_grid,y_grid)
    scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_,
        'best_model'  :clf.best_estimator_
    })
  print(scores)
  return scores

**K-fold CV**

In [11]:
def runCV(splits, regressor, X1, y1, withHoliday=True):
  cat_cols = ['Functioning Day']
  if withHoliday:
    cat_cols.append('Holiday')
  num_cols = [col for col in list(X1) if col not in cat_cols]
  print("Model --> "+ type(regressor()).__name__)
  RMSEscores = []
  R2Score = []
  trainedModels = []
  inputTransformer = []

  kf = KFold(n_splits=splits)
  for train, test in kf.split(X1):
    X_train, X_test, y_train, y_test = X1.loc[train], X1.loc[test], y1.loc[train], y1.loc[test]

    num_pipeline = Pipeline([('scaler', StandardScaler())])
    cat_pipeline = Pipeline([('encoder', OneHotEncoder())])
    full_pipeline = ColumnTransformer([('num', num_pipeline, num_cols),
                                  ('cat', cat_pipeline, cat_cols)])

    #Data preparation
    X_train_prepared = full_pipeline.fit_transform(X_train)
    X_test_prepared = full_pipeline.transform(X_test)

    #Model Fitting
    model = regressor()
    model.fit(X_train_prepared, y_train)

    #Evaluation
    y_test_predict = model.predict(X_test_prepared)
    RMSE = mean_squared_error(y_test, y_test_predict, squared=False)
    R2 = r2_score(y_test,y_test_predict, multioutput='variance_weighted')
    RMSEscores.append(RMSE)
    R2Score.append(R2)
    trainedModels.append(model)
    inputTransformer.append(full_pipeline)
  print("RMSE scores --> " +  str(RMSEscores))
  print("R2 scores  --> " +  str(R2Score))
  printmd("**Min RMSE = " + str(min(RMSEscores)) + "**")
  printmd("**MAX R2 = " + str(max(R2Score)) + "**")
  metric = min(RMSEscores)
  print('=========================================================================================')
  modelIndex = RMSEscores.index(min(RMSEscores))#Based on MSE, could be based on R2 too
  return trainedModels[modelIndex], inputTransformer[modelIndex], metric

In [12]:
def evaluateCVModel(modelPipeline, X1, y1):
  model = modelPipeline[0]
  pipeline = modelPipeline[1]

  #Data preparation
  X_test_prepared = pipeline.transform(X1)

  y_test_predict = model.predict(X_test_prepared)
  RMSE = mean_squared_error(y1, y_test_predict, squared=False)
  R2 = r2_score(y1, y_test_predict, multioutput='variance_weighted')
  printmd("**Model : "+type(model).__name__+",  RMSE = " +str(RMSE)+ ", R2 ="+ str(R2) + "**")

  return y_test_predict, y1

In [13]:
def getFinalMetrics(y_test_predict, y1, model=None):
  RMSE = mean_squared_error(y1, y_test_predict, squared=False)
  R2 = r2_score(y1, y_test_predict, multioutput='variance_weighted')
  if model != None:
    printmd("**Model : "+type(model).__name__+",  RMSE = " +str(RMSE)+ ", R2 ="+ str(R2) + "**")
  else:
    printmd("**RMSE = " +str(RMSE)+ ", R2 ="+ str(R2) + "**")

**Initial Data Split into Train and Test, no changes to be made further**

In [103]:
trainDf = pd.DataFrame()
testDf = pd.DataFrame()
def initData():
  global trainDf, testDf
  X, y = prepareData(df, False)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

  trainDf = X_train
  trainDf['Rented Bike Count'] = y_train

  testDf = X_test
  testDf['Rented Bike Count'] = y_test

In [104]:
initData()

In [105]:
LinearRegressionModels = ["LinearRegression", "RidgeCV", "Lasso", "ElasticNet"]
models = [LinearRegression, RidgeCV, Lasso, KNeighborsRegressor, SGDRegressor, ElasticNet]

# Basic pipeline with SimpleImputer and Linear Regression

In [106]:
X_train, y_train = prepareData(trainDf)
X_test, y_test = prepareData(testDf)
cat_cols = ['Holiday', 'Functioning Day']
num_cols = [col for col in list(X_train) if col not in cat_cols]

strategies = ["mean", "median"] #Currently required only for numerical missing (Temperature)

for strategy in strategies:
    num_pipeline = Pipeline([('imputer', SimpleImputer()),
                            ('scaler', StandardScaler())])
    cat_pipeline = Pipeline([('encoder', OneHotEncoder())])
    full_pipeline = ColumnTransformer([('num', num_pipeline, num_cols),
                                    ('cat', cat_pipeline, cat_cols)])

    #Data preparation
    X_train_prepared = full_pipeline.fit_transform(X_train)
    X_test_prepared = full_pipeline.transform(X_test)

    #Model Fitting
    lin_reg = LinearRegression()
    lin_reg.fit(X_train_prepared, y_train)

    #Evaluation
    y_test_predict = lin_reg.predict(X_test_prepared)
    RMSE = mean_squared_error(y_test, y_test_predict, squared=False)
    R2 = r2_score(y_test,y_test_predict, multioutput='variance_weighted')
    print("Impute strategy - SimpleImputer(" + strategy + "), RMSE = " +str(RMSE)+ ", R2 =", str(R2))

Impute strategy - SimpleImputer(mean), RMSE = 463.25885180035255, R2 = 0.4849137125256828
Impute strategy - SimpleImputer(median), RMSE = 463.25885180035255, R2 = 0.4849137125256828


# Fill Training Data with Imputer (Regressor Model)

In [107]:
def performKNN(dfWithTemp, xcols):
  X, y = prepareData(dfWithTemp)
  Xtem = X[xcols]
  ytem = X['Temperature(C)']

  X_trainT, X_testT, y_trainT, y_testT = train_test_split(Xtem, ytem, test_size= 0.2, random_state=42)
  lin_reg = KNeighborsRegressor()
  lin_reg.fit(X_trainT,y_trainT)
  y_test_predict = lin_reg.predict(X_testT)

  RMSE = mean_squared_error(y_testT,y_test_predict,squared=False)

  R2 = r2_score(y_testT,y_test_predict,multioutput='variance_weighted')

  print("Imputer Model KNN \nRMSE = " +str(RMSE)+ "\nR2 =", str(R2))

  return lin_reg, RMSE, R2

In [108]:
# Use only the rows with temperature value for building the model
dfWithTemp = trainDf.loc[trainDf['Temperature(C)'].notnull()]
# Keep the rows without Temperature value for prediction using the model created
dfPrediction = trainDf.loc[trainDf['Temperature(C)'].isna()]
xparams = ["Dew point temperature(C)", "Solar Radiation (MJ/m2)", "Snowfall (cm)", "Humidity(%)", "Rainfall(mm)",  "Hour_sin", "Hour_cos"]

#Training the Imputer
lin_reg, RMSE, R2 = performKNN(dfWithTemp, xparams)
#Predicting Missing Values
trainDf.loc[trainDf['Temperature(C)'].isna(), "Temperature(C)"] = lin_reg.predict(trainDf.loc[trainDf['Temperature(C)'].isna()][xparams])
testDf.loc[testDf['Temperature(C)'].isna(), "Temperature(C)"] = lin_reg.predict(testDf.loc[testDf['Temperature(C)'].isna()][xparams])

Imputer Model KNN 
RMSE = 0.5042309276712973
R2 = 0.9982082398101662


# Training on complete data with imputation

In [110]:
trainDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7008 entries, 8415 to 7270
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      7008 non-null   datetime64[ns]
 1   Hour                      7008 non-null   int64         
 2   Temperature(C)            7008 non-null   float64       
 3   Humidity(%)               7008 non-null   int64         
 4   Wind speed (m/s)          7008 non-null   float64       
 5   Visibility (10m)          7008 non-null   int64         
 6   Dew point temperature(C)  7008 non-null   float64       
 7   Solar Radiation (MJ/m2)   7008 non-null   float64       
 8   Rainfall(mm)              7008 non-null   float64       
 9   Snowfall (cm)             7008 non-null   float64       
 10  Seasons                   7008 non-null   int64         
 11  Holiday                   7008 non-null   object        
 12  Functioning Day  

In [111]:
#Complete Data
trainDf.reset_index(drop=True, inplace=True)
testDf.reset_index(drop=True, inplace=True)
X, y = prepareData(trainDf)
soFarBestMSE = sys.maxsize
finalModelAllData = []
LRfinalModelAllData = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelAllData = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelAllData = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [439.1711789389289, 479.2462481285659, 450.53966912533883, 435.6023093723511, 459.05856631049477, 422.20842898577047, 452.60772026874, 445.71169457979784, 456.9535734542384, 435.64036159480185]
R2 scores  --> [0.5057287279037879, 0.44907574184960297, 0.50720151872786, 0.535288919607892, 0.5314793298295908, 0.5723684073285139, 0.5146284546456898, 0.5288182078842067, 0.4784090226772204, 0.54379753117227]


<span style='color:None'>**Min RMSE = 422.20842898577047**</span>

<span style='color:None'>**MAX R2 = 0.5723684073285139**</span>

Model --> RidgeCV
RMSE scores --> [438.90777553011037, 479.22171019398667, 450.7763249132104, 435.77623966339354, 459.1674372248913, 422.4129320963961, 452.56828294691365, 445.3948403706469, 456.633202877057, 435.68050400706863]
R2 scores  --> [0.5063214521128895, 0.44913215626025416, 0.5066836764582903, 0.5349177394356518, 0.5312570735594215, 0.5719540471961595, 0.5147130352616969, 0.5294878913098675, 0.47914014213589795, 0.5437134530521917]


<span style='color:None'>**Min RMSE = 422.4129320963961**</span>

...

<span style='color:None'>**Min RMSE = 423.1452842677625**</span>

<span style='color:None'>**MAX R2 = 0.5704685238841181**</span>

Model --> ElasticNet
RMSE scores --> [468.00259645088045, 505.47272024492577, 490.36352490306547, 481.01481959767443, 506.38857228048994, 469.8363961690102, 484.4804080936559, 482.9086174684468, 484.4194973659171, 483.2945695993597]
R2 scores  --> [0.43870104770437934, 0.3871278358148367, 0.4162329129738471, 0.43334386611995934, 0.42988777133683637, 0.47044716519191915, 0.4438616604014727, 0.44689147532364093, 0.4138225007916071, 0.4385316989776946]


<span style='color:None'>**Min RMSE = 468.00259645088045**</span>

<span style='color:None'>**MAX R2 = 0.47044716519191915**</span>

**Testing Full models on unseen data**

In [112]:
#Full Data
print("Evaluation for Model trained on complete data")

X, y = prepareData(testDf)
out = evaluateCVModel(LRfinalModelAllData, X, y)
out = evaluateCVModel(finalModelAllData, X, y)

Evaluation for Model trained on complete data


<span style='color:None'>**Model : LinearRegression,  RMSE = 462.18758667855525, R2 =0.48729318556997747**</span>

<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 304.34946249276453, R2 =0.7776804710175861**</span>

# Experiment with separate models for each seasons

In [117]:
seasonwiseData = {}
seasons = ["Summer", "Spring", "Autumn", "Winter"]
seasontoIdmap = dict()
seasontoIdmap['Summer'] = 0
seasontoIdmap['Spring'] = 1
seasontoIdmap['Autumn'] = 2
seasontoIdmap['Winter'] = 3

mask = (trainDf['Seasons'] == seasontoIdmap['Summer'] )
seasonwiseData["Summer"] = trainDf[mask].copy()

mask = (trainDf['Seasons'] == seasontoIdmap['Spring'])
seasonwiseData["Spring"] = trainDf[mask].copy()

mask = (trainDf['Seasons'] == seasontoIdmap['Autumn'])
seasonwiseData["Autumn"] = trainDf[mask].copy()

mask = (trainDf['Seasons'] == seasontoIdmap['Winter'])
seasonwiseData["Winter"] = trainDf[mask].copy()

finalModelSeasonwise = dict()
LRfinalModelSeasonwise = dict()

for season in seasons:
  seasonwiseData[season].reset_index(drop=True, inplace=True)

In [119]:
for season in seasons:
  printmd("**Season : "+season+"**")
  X, y = prepareData(seasonwiseData[season])
  soFarBestMSE = sys.maxsize
  finalModelWithCurrentSeason = []
  LRsoFarBestMSE = sys.maxsize
  LRfinalModelWithCurrentSeason = []
  for model in models:
    bestModelMSE, imputer, metric = runCV(10, model, X, y)
    if metric < soFarBestMSE:
      finalModelWithCurrentSeason = [bestModelMSE, imputer]
      soFarBestMSE = metric
    if metric < LRsoFarBestMSE:
      if type(model()).__name__ in LinearRegressionModels:
        LRsoFarBestMSE = metric
        LRfinalModelWithCurrentSeason = [bestModelMSE, imputer]
  finalModelSeasonwise[season] = finalModelWithCurrentSeason
  LRfinalModelSeasonwise[season] = LRfinalModelWithCurrentSeason

<span style='color:None'>**Season : Summer**</span>

Model --> LinearRegression
RMSE scores --> [489.1118084504191, 489.01414978996934, 498.3022004864068, 506.764309917095, 452.3585381784103, 483.31120417872756, 492.17006751148637, 504.9842119744287, 488.90413682849083, 464.2920188257495]
R2 scores  --> [0.3586702477691577, 0.4462024968638964, 0.49740921600627475, 0.4775233258030411, 0.6051321169464746, 0.5437766550824896, 0.5280506267018943, 0.4652080534624427, 0.47518311070762836, 0.4860515038722108]


<span style='color:None'>**Min RMSE = 452.3585381784103**</span>

<span style='color:None'>**MAX R2 = 0.6051321169464746**</span>

Model --> RidgeCV
RMSE scores --> [489.09495452302576, 488.96303234330713, 498.420164415114, 506.7564456100511, 452.4774611862437, 483.1928523412486, 492.12202074016676, 504.98271265701163, 488.93531981624307, 464.2091158294641]
R2 scores  --> [0.3587144451844083, 0.4463182695251264, 0.49717122949536086, 0.47753954196122855, 0.6049244717048199, 0.5440000650082342, 0.5281427677656683, 0.4652112290931679, 0.4751161614662503, 0.4862350265887448]


...

<span style='color:None'>**Min RMSE = 87.49253237019431**</span>

<span style='color:None'>**MAX R2 = 0.5059323544460539**</span>

Model --> ElasticNet
RMSE scores --> [92.76080123851635, 97.03044399144726, 146.7880979094873, 124.91075684566405, 105.41396551711097, 129.46970114980317, 116.62134489235918, 119.85881389918241, 128.76094174547825, 131.0933772558949]
R2 scores  --> [0.4404701247490524, 0.41715698210149843, 0.2833948023504983, 0.31273492049836116, 0.42609036988794013, 0.3474041130562403, 0.362228038975252, 0.37849604508614687, 0.2932551150944807, 0.30755834283285965]


<span style='color:None'>**Min RMSE = 92.76080123851635**</span>

<span style='color:None'>**MAX R2 = 0.4404701247490524**</span>

**Testing 4 models on unseen data**

In [120]:
mask = (testDf['Seasons'] == seasontoIdmap["Summer"])
seasonwiseData["Summer"] = testDf[mask].copy()

mask = (testDf['Seasons'] == seasontoIdmap["Spring"])
seasonwiseData["Spring"] = testDf[mask].copy()

mask = (testDf['Seasons'] == seasontoIdmap["Autumn"])
seasonwiseData["Autumn"] = testDf[mask].copy()

mask = (testDf['Seasons'] == seasontoIdmap["Winter"])
seasonwiseData["Winter"] = testDf[mask].copy()

for season in seasons:
  seasonwiseData[season].reset_index(drop=True, inplace=True)

LRPredicted = []
LRground_truth = []
predicted = []
ground_truth = []

for season in seasons:
  printmd("**Season : "+season+"**")
  X, y = prepareData(seasonwiseData[season])
  LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelSeasonwise[season], X, y)
  LRPredicted.extend(LRy_test_predict)
  LRground_truth.extend(LRy1)
  y_test_predict, y1 = evaluateCVModel(finalModelSeasonwise[season], X, y)
  predicted.extend(y_test_predict)
  ground_truth.extend(y1)

#Metrics On full testset
printmd("**Final Results on complete testset**")

print('Linear Models Best output')
getFinalMetrics(LRPredicted, LRground_truth)

print("Overall Best Model")
getFinalMetrics(predicted, ground_truth)

<span style='color:None'>**Season : Summer**</span>

<span style='color:None'>**Model : LinearRegression,  RMSE = 512.5388134178594, R2 =0.4651888366836193**</span>

<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 358.4742543240949, R2 =0.7383848526896162**</span>

<span style='color:None'>**Season : Spring**</span>

<span style='color:None'>**Model : Lasso,  RMSE = 440.81664461845736, R2 =0.5241599556354878**</span>

...

<span style='color:None'>**Final Results on complete testset**</span>

Linear Models Best output


<span style='color:None'>**RMSE = 407.25625620224736, R2 =0.6019221027217457**</span>

Overall Best Model


<span style='color:None'>**RMSE = 303.2186357991382, R2 =0.7793294820047562**</span>

# Experiment with separate models for rain/snowfall or not


In [121]:
threshold = 0.0
mask = (trainDf['Rainfall(mm)'] != threshold) | (trainDf['Snowfall (cm)'] != threshold)
dfWithPrecipitation = trainDf[mask].copy()
dfWithoutPrecipitation = trainDf[~mask].copy()
dfWithPrecipitation.reset_index(drop=True, inplace=True)
dfWithoutPrecipitation.reset_index(drop=True, inplace=True)

**Train Model With Precipitation**

In [122]:
X, y = prepareData(dfWithPrecipitation)
soFarBestMSE = sys.maxsize
finalModelWithPrecipitation = []
LRfinalModelWithPrecipitation = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithPrecipitation = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithPrecipitation = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [147.0402268314847, 181.30817585103537, 139.19182285098768, 154.5470458693383, 247.86026906553093, 167.448787410823, 253.38441119779205, 133.0908722802346, 188.41134618517057, 218.30932191855163]
R2 scores  --> [0.14148281339329205, 0.23862157910898485, 0.2960043119557484, 0.23393961264254134, 0.24159774887313856, 0.32416334768315347, 0.18409610529303844, 0.04236865699208481, 0.2231676684213921, 0.272280273956903]


<span style='color:None'>**Min RMSE = 133.0908722802346**</span>

<span style='color:None'>**MAX R2 = 0.32416334768315347**</span>

Model --> RidgeCV
RMSE scores --> [146.95509009302648, 180.53518175157978, 138.6953807624276, 154.91787701916627, 247.6778907622872, 168.04232244782526, 255.10939956994446, 131.62732278037768, 187.7039087537209, 219.0591160791476]
R2 scores  --> [0.1424766937033909, 0.24509990170321894, 0.30101710409925286, 0.23025892276065718, 0.24271341964200233, 0.3193637470233638, 0.17294928334047022, 0.06331426722566902, 0.2289903383382277, 0.26727291091701166]


<span style='color:None'>**Min RMSE = 131.62732278037768**</span>

...

<span style='color:None'>**Min RMSE = 129.3516920674838**</span>

<span style='color:None'>**MAX R2 = 0.2787955596169919**</span>

Model --> ElasticNet
RMSE scores --> [145.3657876703441, 186.85616732626818, 143.53438893761853, 155.52860351869103, 260.02816305834506, 177.54854893605574, 265.6556266355809, 125.50143242294504, 193.97729185572385, 232.67594609867925]
R2 scores  --> [0.16092442843970656, 0.19131263047896319, 0.2513919656345618, 0.22417792147980842, 0.1653074144689135, 0.24017779423400246, 0.1031552737525796, 0.14847152656471907, 0.17659221090162347, 0.17334830047732197]


<span style='color:None'>**Min RMSE = 125.50143242294504**</span>

<span style='color:None'>**MAX R2 = 0.2513919656345618**</span>

**Train Model Without Precipitation**

In [123]:
X, y = prepareData(dfWithoutPrecipitation)
soFarBestMSE = sys.maxsize
finalModelWithoutPrecipitation = []
LRfinalModelWithoutPrecipitation = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithoutPrecipitation = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithoutPrecipitation = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [437.0040712127694, 485.77530333849285, 458.0541628502339, 435.43919128001096, 463.27911496110124, 426.8138898195972, 445.81042053547577, 450.29241396046064, 456.77710365159965, 435.5154083390599]
R2 scores  --> [0.5187231893847878, 0.4471244835631626, 0.4979936900075952, 0.5428006003341916, 0.5279009777030566, 0.5684326285097613, 0.5385434185009612, 0.5281283193959758, 0.4965780039296772, 0.5424293911645569]


<span style='color:None'>**Min RMSE = 426.8138898195972**</span>

<span style='color:None'>**MAX R2 = 0.5684326285097613**</span>

Model --> RidgeCV
RMSE scores --> [436.76367558232744, 485.72765798508544, 458.15013020490886, 435.51376185030654, 463.4046213615093, 426.90415476205806, 445.7341595060704, 450.2316495270861, 456.54571588767067, 435.7487994205825]
R2 scores  --> [0.5192525438121203, 0.4472329314705257, 0.4977833163521199, 0.542643992752771, 0.5276451514726865, 0.5682500687377572, 0.5387012799702455, 0.5282556636717277, 0.49708790771078015, 0.5419388389949589]


<span style='color:None'>**Min RMSE = 426.90415476205806**</span>

...

<span style='color:None'>**Min RMSE = 428.79607923076236**</span>

<span style='color:None'>**MAX R2 = 0.5644147897694434**</span>

Model --> ElasticNet
RMSE scores --> [472.635954671029, 513.6486856571614, 498.38644492641083, 485.01213341181096, 511.8634343820717, 476.05763557634197, 483.68167403178643, 488.4468809624668, 491.22237064732514, 486.5081103418957]
R2 scores  --> [0.4370400705370343, 0.3818571401298976, 0.4056969610060337, 0.4327743927133533, 0.42369038000993486, 0.46310348541165713, 0.4568126102701896, 0.44477458889420696, 0.41778980251635955, 0.42900640028505954]


<span style='color:None'>**Min RMSE = 472.635954671029**</span>

<span style='color:None'>**MAX R2 = 0.46310348541165713**</span>

**Testing 2 models on unseen data**

In [124]:
threshold = 0.0
mask = (testDf['Rainfall(mm)'] != threshold) | (testDf['Snowfall (cm)'] != threshold)
dfWithPrecipitation = testDf[mask].copy()
dfWithoutPrecipitation = testDf[~mask].copy()
dfWithPrecipitation.reset_index(drop=True, inplace=True)
dfWithoutPrecipitation.reset_index(drop=True, inplace=True)

predicted = []
ground_truth = []

LRpredicted = []
LRground_truth = []


#WithPrecipitation
print("Evaluation for Model With Precipitation")
X, y = prepareData(dfWithPrecipitation)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithPrecipitation, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithPrecipitation, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#WithoutPrecipitation
print("Evaluation for Model Without Precipitation")
X, y = prepareData(dfWithoutPrecipitation)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithoutPrecipitation, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithoutPrecipitation, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#Metrics On full testset
#Metrics On full testset
printmd("**Final Results on complete testset**")

print('Linear Models Best output')
getFinalMetrics(LRpredicted, LRground_truth)

print("Overall Best Model")
getFinalMetrics(predicted, ground_truth)

Evaluation for Model With Precipitation


<span style='color:None'>**Model : ElasticNet,  RMSE = 241.51041994964865, R2 =0.12915737092146784**</span>

<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 246.05048040413138, R2 =0.09610836200155148**</span>

Evaluation for Model Without Precipitation


<span style='color:None'>**Model : LinearRegression,  RMSE = 463.7263030265346, R2 =0.4883937140686007**</span>

...

<span style='color:None'>**Final Results on complete testset**</span>

Linear Models Best output


<span style='color:None'>**RMSE = 445.72738430488687, R2 =0.5231616581683753**</span>

Overall Best Model


<span style='color:None'>**RMSE = 310.9431288940032, R2 =0.7679431117915152**</span>

In [90]:
#End

# Experiment with separate models for Weekday and Weekend data 

In [125]:
trainDf['datetime'] = pd.to_datetime(trainDf['Date'])
trainDf['day'] = trainDf['datetime'].dt.dayofweek# 0 - Monday

testDf['datetime'] = pd.to_datetime(testDf['Date'])
testDf['day'] = testDf['datetime'].dt.dayofweek# 0 - Monday

In [126]:
threshold = 0.0
mask = (trainDf['day'] > 4)

dfWithWeekdays = trainDf[mask].copy()
dfWithWeekdays = dfWithWeekdays.drop(['datetime'], axis=1)
dfWithWeekdays = dfWithWeekdays.drop(['day'], axis=1)

dfWithWeekends = trainDf[~mask].copy()
dfWithWeekends = dfWithWeekends.drop(['datetime'], axis=1)
dfWithWeekends = dfWithWeekends.drop(['day'], axis=1)

dfWithWeekdays.reset_index(drop=True, inplace=True)
dfWithWeekends.reset_index(drop=True, inplace=True)

**Train model with Weekdays**

In [127]:
X, y = prepareData(dfWithWeekdays, 4)
soFarBestMSE = sys.maxsize
finalModelWithWeekdays = []
LRfinalModelWithWeekdays = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithWeekdays = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithWeekdays = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [366.9630922513392, 420.9455731493893, 368.21001885634587, 397.56588159415173, 373.7821351532544, 371.86820204177536, 409.9213356980916, 435.8147431639859, 404.18965210750673, 376.5550602419459]
R2 scores  --> [0.583996286477338, 0.54217190401113, 0.6612475518309722, 0.5929043407968502, 0.6151783295982615, 0.6414193597501914, 0.6063298004686608, 0.5945369105218341, 0.5361250099440565, 0.5815857502314202]


<span style='color:None'>**Min RMSE = 366.9630922513392**</span>

<span style='color:None'>**MAX R2 = 0.6612475518309722**</span>

Model --> RidgeCV
RMSE scores --> [365.23496139635535, 421.3561992983337, 368.8521220094676, 398.83230116137554, 373.8535752545069, 371.51891985139747, 409.8410677140666, 436.96694417900187, 403.6876064324556, 375.05695386254035]
R2 scores  --> [0.5879052140932352, 0.5412782593517003, 0.6600650545609965, 0.5903066579285404, 0.6150312154174389, 0.6420926467446357, 0.6064839569702858, 0.592390161032613, 0.5372766564382079, 0.5849084098798176]


<span style='color:None'>**Min RMSE = 365.23496139635535**</span>

...

<span style='color:None'>**Min RMSE = 367.7200490723247**</span>

<span style='color:None'>**MAX R2 = 0.6605474585223605**</span>

Model --> ElasticNet
RMSE scores --> [388.9030784849507, 452.8146073069958, 414.22342467117795, 446.2688520826244, 405.6499742421979, 397.11354460580355, 455.4133652015115, 490.0567875242665, 427.23875232936734, 401.21137460015336]
R2 scores  --> [0.5327651850091191, 0.4702250682682454, 0.5712930246557224, 0.48705428143306306, 0.5467630119105844, 0.591080189865152, 0.5141043201234547, 0.48732716397485065, 0.48171116554699844, 0.5249974243605489]


<span style='color:None'>**Min RMSE = 388.9030784849507**</span>

<span style='color:None'>**MAX R2 = 0.591080189865152**</span>

**Train model with Weekends Data**

In [128]:
X, y = prepareData(dfWithWeekends)
soFarBestMSE = sys.maxsize
finalModelWithWeekends = []
LRfinalModelWithWeekends = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithWeekends = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithWeekends = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [464.2439154617931, 499.8369944321409, 473.52747435809056, 442.10613059986264, 492.275954662797, 445.4422278980246, 457.16531291388407, 448.8152181262841, 480.3775167860583, 452.79264374278375]
R2 scores  --> [0.48045138122385805, 0.41429009209948187, 0.46509248783956814, 0.5113762008988949, 0.5094560111508495, 0.5447987005607539, 0.49170910585974387, 0.49839712430299954, 0.4636869253504916, 0.5332273827289393]


<span style='color:None'>**Min RMSE = 442.10613059986264**</span>

<span style='color:None'>**MAX R2 = 0.5447987005607539**</span>

Model --> RidgeCV
RMSE scores --> [464.31095093943355, 499.85612573987197, 473.82412865818884, 442.0781719048002, 492.68978315754276, 445.6919538352376, 457.1603896966038, 447.9385058736406, 479.5394543900576, 453.1313790390308]
R2 scores  --> [0.4803013277575573, 0.41424525503836873, 0.46442206287892734, 0.5114379998692573, 0.508630919434141, 0.5442881632624431, 0.4917200533790259, 0.5003548648432383, 0.46555658718660503, 0.5325287339982185]


<span style='color:None'>**Min RMSE = 442.0781719048002**</span>

...

<span style='color:None'>**Min RMSE = 442.152070684341**</span>

<span style='color:None'>**MAX R2 = 0.543093393476425**</span>

Model --> ElasticNet
RMSE scores --> [498.1860733074405, 522.4531525658981, 513.4107577896482, 482.74167974511204, 548.6778961270813, 496.4843537156063, 489.5523246809509, 480.3277375409118, 511.1886797401956, 507.7046027813918]
R2 scores  --> [0.40170287047184466, 0.36008765864317616, 0.37119170546465363, 0.4174259493098874, 0.3906095534877567, 0.4345009822489183, 0.4171403092641689, 0.4254865475009672, 0.39268292087398615, 0.41314765134143205]


<span style='color:None'>**Min RMSE = 480.3277375409118**</span>

<span style='color:None'>**MAX R2 = 0.4345009822489183**</span>

**Test 2 models on unseen data**

In [129]:
mask = (testDf['day'] > 4)

dfWithWeekdays = testDf[mask].copy()
dfWithWeekdays = dfWithWeekdays.drop(['datetime'], axis=1)
dfWithWeekdays = dfWithWeekdays.drop(['day'], axis=1)

dfWithWeekends = testDf[~mask].copy()
dfWithWeekends = dfWithWeekends.drop(['datetime'], axis=1)
dfWithWeekends = dfWithWeekends.drop(['day'], axis=1)

testDf = testDf.drop(['datetime'], axis=1)
testDf = testDf.drop(['day'], axis=1)

trainDf = trainDf.drop(['datetime'], axis=1)
trainDf = trainDf.drop(['day'], axis=1)

dfWithWeekdays.reset_index(drop=True, inplace=True)
dfWithWeekends.reset_index(drop=True, inplace=True)

predicted = []
ground_truth = []

LRpredicted = []
LRground_truth = []

#WithPrecipitation
print("Evaluation for Model With Weekdays")
X, y = prepareData(dfWithWeekdays)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithWeekdays, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithWeekdays, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#WithoutPrecipitation
print("Evaluation for Model With Weekends")
X, y = prepareData(dfWithWeekends)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithWeekends, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithWeekends, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#Metrics On full testset
print("")
printmd("**Final Results on complete testset**")

print('Linear Models Best output')
getFinalMetrics(LRpredicted, LRground_truth)

print("Overall Best Model")
getFinalMetrics(predicted, ground_truth)

Evaluation for Model With Weekdays


<span style='color:None'>**Model : Lasso,  RMSE = 394.2163303586817, R2 =0.5918083312056913**</span>

<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 251.07267407399527, R2 =0.8344253632664903**</span>

Evaluation for Model With Weekends


<span style='color:None'>**Model : Lasso,  RMSE = 482.19801661349175, R2 =0.45927144029412503**</span>

...

<span style='color:None'>**Final Results on complete testset**</span>

Linear Models Best output


<span style='color:None'>**RMSE = 459.7709591400728, R2 =0.4926407207437466**</span>

Overall Best Model


<span style='color:None'>**RMSE = 296.73992521933764, R2 =0.7886586383127328**</span>

# Experiment with separate models for Holiday & Not Holiday

In [130]:
mask = (trainDf['Holiday'] == 'Holiday')

dfWithHolidays = trainDf[mask].copy()
dfWithHolidays = dfWithHolidays.drop(['Holiday'], axis=1)

dfWithNonHolidays = trainDf[~mask].copy()
dfWithNonHolidays = dfWithNonHolidays.drop(['Holiday'], axis=1)

dfWithHolidays.reset_index(drop=True, inplace=True)
dfWithNonHolidays.reset_index(drop=True, inplace=True)

**Train model with Holidays**

In [131]:
X, y = prepareData(dfWithHolidays)
soFarBestMSE = sys.maxsize
LRsoFarBestMSE = sys.maxsize
finalModelWithHolidays = []
LRfinalModelWithHolidays = []
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y, withHoliday=False)
  if metric < soFarBestMSE:
    finalModelWithHolidays = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRfinalModelWithHolidays = [bestModelMSE, imputer]
      LRsoFarBestMSE = metric

Model --> LinearRegression
RMSE scores --> [354.08172923927236, 349.0562748935887, 378.11598287771074, 310.89593306962274, 361.90375217236397, 235.57733963390908, 294.8590307456816, 337.4478162036861, 332.01552994244537, 331.0316164817982]
R2 scores  --> [0.6383846367411903, 0.6439383337272293, 0.6085603541457353, 0.7462357155633699, 0.6429565178734433, 0.6544190668330977, 0.7473461961338138, 0.6120015584320712, 0.5231083103641838, 0.5739797116467484]


<span style='color:None'>**Min RMSE = 235.57733963390908**</span>

<span style='color:None'>**MAX R2 = 0.7473461961338138**</span>

Model --> RidgeCV
RMSE scores --> [360.82941898411445, 348.3377646565596, 381.41151093805655, 312.23368848884, 363.72136656801695, 232.8561373768817, 300.9376007892196, 333.74839782097837, 330.8442824952811, 325.3928146084655]
R2 scores  --> [0.6244707964737672, 0.6454026858195321, 0.6017073143707394, 0.7440471703059393, 0.6393611030921476, 0.6623567085882756, 0.736821818015381, 0.6204621296534373, 0.5264670265330644, 0.588369781570576]


<span style='color:None'>**Min RMSE = 232.8561373768817**</span>

...

<span style='color:None'>**Min RMSE = 228.08288053501488**</span>

<span style='color:None'>**MAX R2 = 0.7492329482929483**</span>

Model --> ElasticNet
RMSE scores --> [407.64297780271767, 368.27092490526695, 401.66939107809213, 430.68350202745756, 443.525470852411, 277.1392834058187, 394.14968381861814, 353.29087057675673, 323.7373773927081, 321.13338711140756]
R2 scores  --> [0.5207084537798206, 0.6036588312277885, 0.5582747603102012, 0.5130135384150923, 0.4637441178839396, 0.5217236278159492, 0.5485399357639187, 0.5747135291816845, 0.5465925556667515, 0.5990758174087513]


<span style='color:None'>**Min RMSE = 277.1392834058187**</span>

<span style='color:None'>**MAX R2 = 0.6036588312277885**</span>

**Train model with Non-Holiday Data**

In [132]:
X, y = prepareData(dfWithNonHolidays)
soFarBestMSE = sys.maxsize
finalModelWithNonHolidays = []
LRsoFarBestMSE = sys.maxsize
LRfinalModelWithNonHolidays = []
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y, withHoliday=False)
  if metric < soFarBestMSE:
    finalModelWithNonHolidays = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRfinalModelWithNonHolidays = [bestModelMSE, imputer]
      LRsoFarBestMSE = metric

Model --> LinearRegression
RMSE scores --> [441.05541605960065, 485.4414403365998, 453.25482494569474, 439.25448164141, 464.1918613662689, 427.17030531068013, 451.4888475769772, 452.3720923736406, 462.25956564381863, 439.6637539555557]
R2 scores  --> [0.49898305872401705, 0.43620143464375793, 0.4968939824748362, 0.5278340585815195, 0.5273873619041881, 0.5670438322934015, 0.510845337964709, 0.5286986355530862, 0.4747862821977905, 0.5416274781932754]


<span style='color:None'>**Min RMSE = 427.17030531068013**</span>

<span style='color:None'>**MAX R2 = 0.5670438322934015**</span>

Model --> RidgeCV
RMSE scores --> [440.7903886191445, 485.4465320067661, 453.5635490133974, 439.4388738110826, 464.14512765618355, 427.4141881017338, 451.46995702632483, 452.09309646823925, 461.87197360957094, 439.6225244706494]
R2 scores  --> [0.4995849937119107, 0.43618960750601743, 0.4962083909519178, 0.5274375596042098, 0.5274825201007001, 0.5665493189834259, 0.5108862701279631, 0.5292797969694917, 0.47566666769996346, 0.5417134419630756]


<span style='color:None'>**Min RMSE = 427.4141881017338**</span>

...

<span style='color:None'>**Min RMSE = 427.90234340114125**</span>

<span style='color:None'>**MAX R2 = 0.5655586542679333**</span>

Model --> ElasticNet
RMSE scores --> [467.29435994489086, 510.2554693380713, 493.7850988122922, 480.55656431116125, 509.44293107379616, 473.88933154270296, 480.2426523207184, 490.2790463642004, 489.47898583107286, 487.899785877513]
R2 scores  --> [0.4375975928113084, 0.3770895622885576, 0.40289514857141917, 0.43486620013271626, 0.4307521902672503, 0.46716139334550033, 0.44655610555154934, 0.44640296801944734, 0.4111124808506492, 0.43553307093950333]


<span style='color:None'>**Min RMSE = 467.29435994489086**</span>

<span style='color:None'>**MAX R2 = 0.46716139334550033**</span>

**Test 2 models on unseen data**

In [133]:
mask = (testDf['Holiday'] == 'Holiday')

dfWithHolidays = testDf[mask].copy()
dfWithHolidays = dfWithHolidays.drop(['Holiday'], axis=1)

dfWithNonHolidays = testDf[~mask].copy()
dfWithNonHolidays = dfWithNonHolidays.drop(['Holiday'], axis=1)

dfWithHolidays.reset_index(drop=True, inplace=True)
dfWithNonHolidays.reset_index(drop=True, inplace=True)

predicted = []
ground_truth = []

LRpredicted = []
LRground_truth = []

#With Holiday
print("Evaluation for Model With Holidays")
X, y = prepareData(dfWithHolidays)
y_test_predict, y1 = evaluateCVModel(finalModelWithHolidays, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithHolidays, X, y)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#With Non-Holiday
print("Evaluation for Model With Non-Holidays")
X, y = prepareData(dfWithNonHolidays)
y_test_predict, y1 = evaluateCVModel(finalModelWithNonHolidays, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithNonHolidays, X, y)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#Metrics On full testset
printmd("**Final Results on complete testset**")

print('Linear Models Best output')
getFinalMetrics(LRpredicted, LRground_truth)

print("Overall Best Model")
getFinalMetrics(predicted, ground_truth)

Evaluation for Model With Holidays


<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 180.69184403287272, R2 =0.9082499549116151**</span>

<span style='color:None'>**Model : RidgeCV,  RMSE = 335.13382924217996, R2 =0.684379121837787**</span>

Evaluation for Model With Non-Holidays


<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 310.66842664775174, R2 =0.7693802176284472**</span>

...

<span style='color:None'>**Final Results on complete testset**</span>

Linear Models Best output


<span style='color:None'>**RMSE = 459.2775128858522, R2 =0.49372917657251547**</span>

Overall Best Model


<span style='color:None'>**RMSE = 305.9389583817461, R2 =0.7753522346187448**</span>

In [134]:
#End

# Experiment with separate models for Day-time & Night-time

In [135]:
day = (trainDf['Hour'] >= 7) & (trainDf['Hour'] <= 18) # 7AM to 6PM
night = ((trainDf['Hour'] >= 0) & (trainDf['Hour'] <= 6)) | ((trainDf['Hour'] >= 19) & (trainDf['Hour'] <= 23)) #7PM to 6AM
trainDay = trainDf[day].copy()
trainNight = trainDf[night].copy()

day = (testDf['Hour'] >= 7) & (testDf['Hour'] <= 18) # 7AM to 6PM
night = ((testDf['Hour'] >= 0) & (testDf['Hour'] <= 6)) | ((testDf['Hour'] >= 19) & (testDf['Hour'] <= 23))
testDay = testDf[day].copy()
testNight = testDf[night].copy()

trainDay.reset_index(drop=True, inplace=True)
trainNight.reset_index(drop=True, inplace=True)
testDay.reset_index(drop=True, inplace=True)
testNight.reset_index(drop=True, inplace=True)

**Train model with Day-time Data**

In [136]:
X, y = prepareData(trainDay)
soFarBestMSE = sys.maxsize
finalModelWithDay = []
LRfinalModelWithDay = []
LRsoFarBestMSE = sys.maxsize

for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithDay = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithDay = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [451.22339241038645, 501.73208662535285, 418.7611706847837, 425.7470143536772, 460.27179080326, 401.8652311597946, 446.46856773644157, 452.5181210404397, 429.2623074382998, 407.93631672052254]
R2 scores  --> [0.4916488694536732, 0.4481036383030525, 0.5190155175303388, 0.5611666702785131, 0.5421982699375186, 0.6163853100649411, 0.4967851335322701, 0.4877849727062792, 0.5515754158675628, 0.5648720753162658]


<span style='color:None'>**Min RMSE = 401.8652311597946**</span>

<span style='color:None'>**MAX R2 = 0.6163853100649411**</span>

Model --> RidgeCV
RMSE scores --> [451.1926910077157, 501.685813731268, 418.77982633831675, 425.80531335602655, 460.2051498185242, 401.9994133648931, 446.44852343265785, 452.1587396966953, 429.2718697520481, 408.0599396459662]
R2 scores  --> [0.4917180438895, 0.44820543232795534, 0.518972661218142, 0.5610464801370476, 0.5423308270488632, 0.6161290905435585, 0.49683031640049846, 0.48859823263820884, 0.5515554372938495, 0.564608308961354]


<span style='color:None'>**Min RMSE = 401.9994133648931**</span>

...

<span style='color:None'>**Min RMSE = 402.93864318585565**</span>

<span style='color:None'>**MAX R2 = 0.6143332461757376**</span>

Model --> ElasticNet
RMSE scores --> [486.0120473542857, 545.0443672537904, 464.4473209074055, 480.9175295157063, 531.7001552685736, 466.03832259089603, 485.57269713149697, 488.52844742420973, 493.0719386344177, 464.3403746172496]
R2 scores  --> [0.41024090025391735, 0.3487053794773828, 0.4083414034434447, 0.44006502184783836, 0.3890828928157314, 0.48408563095199786, 0.40477631657053625, 0.40301958782258995, 0.40835053446437286, 0.43622591392364396]


<span style='color:None'>**Min RMSE = 464.3403746172496**</span>

<span style='color:None'>**MAX R2 = 0.48408563095199786**</span>

**Train model with Night-time Data**

In [137]:
X, y = prepareData(trainNight)
soFarBestMSE = sys.maxsize
finalModelWithNight = []
LRfinalModelWithNight = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithNight = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithNight = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [339.05783061470134, 356.30155768497974, 328.256809541209, 343.82855209177586, 345.61123191247236, 342.98591438500847, 340.60946806566847, 331.90965644736036, 369.82235116654164, 347.1848402344603]
R2 scores  --> [0.6845156757933356, 0.6531108689537657, 0.7192760476276208, 0.7018165403856642, 0.703168856320308, 0.6791521227595145, 0.7178083861507212, 0.7401953967585825, 0.6242264682805478, 0.7133367494324162]


<span style='color:None'>**Min RMSE = 328.256809541209**</span>

<span style='color:None'>**MAX R2 = 0.7401953967585825**</span>

Model --> RidgeCV
RMSE scores --> [338.60025547690856, 356.2785308887998, 328.2004714723784, 343.90482925248114, 345.6285611457311, 342.8969161866201, 340.624264383066, 332.1029738113486, 369.7453348799655, 347.6235658699041]
R2 scores  --> [0.6853666242814551, 0.6531557044984455, 0.7193723995551644, 0.7016842237968651, 0.7031390888555331, 0.6793186087702471, 0.7177838684096245, 0.7398926675880595, 0.6243829632486655, 0.7126117985382097]


<span style='color:None'>**Min RMSE = 328.2004714723784**</span>

...

<span style='color:None'>**Min RMSE = 328.28180391661584**</span>

<span style='color:None'>**MAX R2 = 0.739379262239158**</span>

Model --> ElasticNet
RMSE scores --> [369.1463190110432, 390.28895538717137, 373.1508378811343, 403.5142904305999, 391.2007053947612, 363.74358718414976, 391.18722670527717, 377.5865382689215, 383.40324490690574, 399.872836903535]
R2 scores  --> [0.6260381399204131, 0.5837753722785022, 0.6372388050277097, 0.5893067999695873, 0.6196941308901069, 0.6391412077757206, 0.627779745159694, 0.6637672048684686, 0.5961208400770991, 0.6197280237426306]


<span style='color:None'>**Min RMSE = 363.74358718414976**</span>

<span style='color:None'>**MAX R2 = 0.6637672048684686**</span>

**Test 2 models on unseen data**

In [138]:
testDay = testDf[day].copy()
testNight = testDf[night].copy()

predicted = []
ground_truth = []

LRpredicted = []
LRground_truth = []

#With Day-time
print("Evaluation for Model With Day-time")
X, y = prepareData(testDay)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithDay, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithDay, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#With Night-time
print("Evaluation for Model With Night-time")
X, y = prepareData(testNight)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithNight, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithNight, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#Metrics On full testset
printmd("**Final Results on complete testset**")
print('Linear Models Best output')
getFinalMetrics(LRpredicted, LRground_truth)

print("Overall Best Model")
getFinalMetrics(predicted, ground_truth)

Evaluation for Model With Day-time


<span style='color:None'>**Model : LinearRegression,  RMSE = 455.94145042416085, R2 =0.5175410728914348**</span>

<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 371.91530132395064, R2 =0.6789813332500849**</span>

Evaluation for Model With Night-time


<span style='color:None'>**Model : RidgeCV,  RMSE = 342.1207646961955, R2 =0.6744532163537097**</span>

...

<span style='color:None'>**Final Results on complete testset**</span>

Linear Models Best output


<span style='color:None'>**RMSE = 402.1031043699273, R2 =0.6119323975341436**</span>

Overall Best Model


<span style='color:None'>**RMSE = 312.02160664559534, R2 =0.7663305840725535**</span>

# Experiment with separate models based on 6 Hrs Time-slot

*  Offtime - 12AM to 6AM

*  Peaktime - 7AM to 11PM

In [139]:
day = (trainDf['Hour'] >= 7) & (trainDf['Hour'] <= 23) # 7AM to 11PM
night = ((trainDf['Hour'] >= 0) & (trainDf['Hour'] <= 6)) #12AM to 6AM - Offtime
trainPeaktime = trainDf[day].copy()
trainOfftime = trainDf[night].copy()

day = (testDf['Hour'] >= 7) & (testDf['Hour'] <= 23) # 7AM to 11PM
night = ((testDf['Hour'] >= 0) & (testDf['Hour'] <= 6))
testPeaktime = testDf[day].copy()
testOfftime = testDf[night].copy()

trainPeaktime.reset_index(drop=True, inplace=True)
trainOfftime.reset_index(drop=True, inplace=True)
testPeaktime.reset_index(drop=True, inplace=True)
testOfftime.reset_index(drop=True, inplace=True)

**Training Peaktime**

In [140]:
X, y = prepareData(trainPeaktime)
soFarBestMSE = sys.maxsize
finalModelWithPeakTime = []
LRfinalModelWithPeakTime = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithPeakTime = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithPeakTime = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [459.18396690887334, 505.37203332556845, 446.5769521740289, 441.7642857197718, 479.0900300418251, 441.17116862453105, 468.0074648981901, 478.299693057098, 471.4797307826033, 449.8824284765776]
R2 scores  --> [0.5094970609785205, 0.46946074951966943, 0.538708631245443, 0.5748464120831691, 0.5204937942211656, 0.5725080396084145, 0.5224347236270375, 0.5161797573546967, 0.5051837242688909, 0.5610926308521667]


<span style='color:None'>**Min RMSE = 441.17116862453105**</span>

<span style='color:None'>**MAX R2 = 0.5748464120831691**</span>

Model --> RidgeCV
RMSE scores --> [459.1798896176363, 505.3543353536008, 446.64839209154877, 441.8119569514291, 479.02605726957984, 441.251187423446, 467.9941301788469, 478.12221460564706, 471.4873494342911, 449.85477967852]
R2 scores  --> [0.5095057717114732, 0.46949790750925113, 0.538561031808938, 0.5747546496234908, 0.5206218423596521, 0.5723529501651972, 0.5224619373305917, 0.5165387445654863, 0.5051677326447912, 0.5611465777838273]


<span style='color:None'>**Min RMSE = 441.251187423446**</span>

...

<span style='color:None'>**Min RMSE = 442.549852558649**</span>

<span style='color:None'>**MAX R2 = 0.5715123767885142**</span>

Model --> ElasticNet
RMSE scores --> [496.080671047438, 547.3185311398463, 504.08268797447477, 510.89921960498225, 535.0058519278664, 499.4914017380605, 512.182014105167, 522.5769088763017, 515.7602052108282, 519.0368648099541]
R2 scores  --> [0.4275035548280529, 0.377734941410126, 0.4122586005659949, 0.43136299075645557, 0.40203325702542925, 0.4520136514235167, 0.42802660474963067, 0.42245708322472286, 0.4078747481410322, 0.41578699457090595]


<span style='color:None'>**Min RMSE = 496.080671047438**</span>

<span style='color:None'>**MAX R2 = 0.4520136514235167**</span>

**Training Offtime**

In [141]:
X, y = prepareData(trainOfftime)
soFarBestMSE = sys.maxsize
finalModelWithOffTime = []
LRfinalModelWithOffTime = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithOffTime = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithOffTime = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [160.31183828041372, 177.13518199001993, 153.17075947684205, 135.44700510416865, 150.3091801962329, 140.23224777506113, 139.94750328510784, 147.7669716470439, 144.00507129471936, 173.3814774470394]
R2 scores  --> [0.6348879480099341, 0.601837513100748, 0.6966666395878987, 0.6632626957799543, 0.705971489822049, 0.6837829956635088, 0.6715460847874597, 0.6898977339752178, 0.6647528481826932, 0.6337822663291668]


<span style='color:None'>**Min RMSE = 135.44700510416865**</span>

<span style='color:None'>**MAX R2 = 0.705971489822049**</span>

Model --> RidgeCV
RMSE scores --> [160.41284989346795, 177.11014922537032, 153.21051209280654, 135.31362245879535, 150.39171157732508, 140.2274425116442, 139.77674884049162, 147.76997679186618, 143.91256111916186, 173.36830475029]
R2 scores  --> [0.6344276928368717, 0.6019500418873895, 0.6965091701155308, 0.6639255793515485, 0.7056485123284665, 0.6838046665700628, 0.6723471101580323, 0.6898851207141513, 0.6651834415059044, 0.633837911164657]


<span style='color:None'>**Min RMSE = 135.31362245879535**</span>

...

<span style='color:None'>**Min RMSE = 135.0661407953113**</span>

<span style='color:None'>**MAX R2 = 0.7020367913128528**</span>

Model --> ElasticNet
RMSE scores --> [191.00904689172052, 199.12305921577942, 185.31466309825467, 157.61745416400447, 186.96401911261785, 167.9429344133299, 163.02219668565732, 179.8755632964749, 168.98833471043463, 201.46629597383367]
R2 scores  --> [0.48167416552904574, 0.4968542641212931, 0.5559948436965205, 0.5440039985848916, 0.5450805536439209, 0.5464628112563932, 0.5543051508712503, 0.5404904646958824, 0.5383392249789838, 0.5055313344708192]


<span style='color:None'>**Min RMSE = 157.61745416400447**</span>

<span style='color:None'>**MAX R2 = 0.5559948436965205**</span>

**Testing both the models on unseen data**

In [142]:
predicted = []
ground_truth = []

LRpredicted = []
LRground_truth = []

#With Peaktime
print("Evaluation for Model With Peak Time")
X, y = prepareData(testPeaktime)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithPeakTime, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithPeakTime, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#With Offtime
print("Evaluation for Model With Off-time")
X, y = prepareData(testOfftime)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithOffTime, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithOffTime, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#Metrics On full testset
printmd("**Final Results on complete testset**")

print('Linear Models Best output')
getFinalMetrics(LRpredicted, LRground_truth)

print("Overall Best Model")
getFinalMetrics(predicted, ground_truth)

Evaluation for Model With Peak Time


<span style='color:None'>**Model : LinearRegression,  RMSE = 473.1006327032698, R2 =0.5163774483613727**</span>

<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 342.47009814504514, R2 =0.7465777147642559**</span>

Evaluation for Model With Off-time


<span style='color:None'>**Model : RidgeCV,  RMSE = 158.5815978153015, R2 =0.619842505719126**</span>

...

<span style='color:None'>**Final Results on complete testset**</span>

Linear Models Best output


<span style='color:None'>**RMSE = 405.88547786128794, R2 =0.6045973628736554**</span>

Overall Best Model


<span style='color:None'>**RMSE = 294.9021135484493, R2 =0.791268350210807**</span>

# Experiment with separate models based on Different Time-slots

5-8, 9-10, 11-18, 19-4 

In [151]:
model1 = (trainDf['Hour'] >= 5) & (trainDf['Hour'] <= 8) 
model2 = ((trainDf['Hour'] >= 9) & (trainDf['Hour'] <= 10)) 
model3 = (trainDf['Hour'] >= 11) & (trainDf['Hour'] <= 17) 
model4 = ((trainDf['Hour'] >= 18) | (trainDf['Hour'] <= 4)) 
train1 = trainDf[model1].copy() #.drop("Dew point temperature(C)",axis=1)
train2 = trainDf[model2].copy() #.drop("Dew point temperature(C)",axis=1)
train3 = trainDf[model3].copy() #.drop("Dew point temperature(C)",axis=1)
train4 = trainDf[model4].copy() #.drop("Dew point temperature(C)",axis=1)

m1 = (testDf['Hour'] >= 5) & (testDf['Hour'] <= 8) # 7AM to 11PM
m2 = ((testDf['Hour'] >= 9) & (testDf['Hour'] <= 10))
m3 = (testDf['Hour'] >= 11) & (testDf['Hour'] <= 17) # 7AM to 11PM
m4 = ((testDf['Hour'] >= 18) | (testDf['Hour'] <= 4))
test1 = testDf[m1].copy() #.drop("Dew point temperature(C)",axis=1)
test2 = testDf[m2].copy() #.drop("Dew point temperature(C)",axis=1)
test3 = testDf[m3].copy() #.drop("Dew point temperature(C)",axis=1)
test4 = testDf[m4].copy() #.drop("Dew point temperature(C)",axis=1)

train1.reset_index(drop=True, inplace=True)
train2.reset_index(drop=True, inplace=True)
train3.reset_index(drop=True, inplace=True)
train4.reset_index(drop=True, inplace=True)
test1.reset_index(drop=True, inplace=True)
test2.reset_index(drop=True, inplace=True)
test3.reset_index(drop=True, inplace=True)
test4.reset_index(drop=True, inplace=True)

**Training Peaktime**

In [152]:
X, y = prepareData(train1)
soFarBestMSE = sys.maxsize
finalModelWithTime = []
LRfinalModelWithTime = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithTime = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithTime = [bestModelMSE, imputer]

X, y = prepareData(train2)
soFarBestMSE = sys.maxsize
finalModelWithTime2 = []
LRfinalModelWithTime2 = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithTime2 = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithTime2 = [bestModelMSE, imputer]

X, y = prepareData(train3)
soFarBestMSE = sys.maxsize
finalModelWithTime3 = []
LRfinalModelWithTime3 = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithTime3 = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithTime3 = [bestModelMSE, imputer]

X, y = prepareData(train4)
soFarBestMSE = sys.maxsize
finalModelWithTime4 = []
LRfinalModelWithTime4 = []
LRsoFarBestMSE = sys.maxsize
for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelWithTime4 = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelWithTime4 = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [370.42832802071837, 300.7083417113362, 374.826371453365, 333.3382831179428, 350.33122946358793, 354.44312704150246, 390.20417175539944, 405.9365515300386, 376.55356371129005, 394.43421821258096]
R2 scores  --> [0.5373474375674354, 0.6905873159282168, 0.6025290579462672, 0.7008154505071666, 0.6451616341446302, 0.6052054289123854, 0.39895172902901, 0.49756283637626975, 0.5501118692890203, 0.5349881453600271]


<span style='color:None'>**Min RMSE = 300.7083417113362**</span>

<span style='color:None'>**MAX R2 = 0.7008154505071666**</span>

Model --> RidgeCV
RMSE scores --> [370.33703567762785, 300.6660428072252, 374.8870559258107, 333.35921853524184, 350.0293235232429, 354.3928164758414, 389.1833959880983, 405.7233624654674, 376.49539429895674, 396.13473400014954]
R2 scores  --> [0.5375754516184199, 0.6906743563933131, 0.602400346261016, 0.7007778685647225, 0.6457729507592467, 0.6053174972642337, 0.40209230541742513, 0.498090435980261, 0.5502508545987952, 0.5309699108512174]


<span style='color:None'>**Min RMSE = 300.6660428072252**</span>

...

<span style='color:None'>**Min RMSE = 357.3318757674957**</span>

<span style='color:None'>**MAX R2 = 0.7503735746459076**</span>

Model --> ElasticNet
RMSE scores --> [461.5835037831147, 429.9364948709761, 462.4878800361987, 404.87736071240977, 456.8954090667193, 406.32118678874025, 431.83267868276795, 431.6168652575557, 411.78003453320457, 427.4001561014445]
R2 scores  --> [0.6209046802728011, 0.6154240940580742, 0.6208640117971396, 0.6527645371270163, 0.5922650378010235, 0.6772353407795701, 0.6460209018659779, 0.6407897583464008, 0.6469030046618417, 0.6745624093203186]


<span style='color:None'>**Min RMSE = 404.87736071240977**</span>

<span style='color:None'>**MAX R2 = 0.6772353407795701**</span>

**Training Offtime**

**Testing both the models on unseen data**

In [153]:
predicted = []
ground_truth = []

LRpredicted = []
LRground_truth = []

#With Peaktime
print("Evaluation for Model With Time 1")
X, y = prepareData(test1)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithTime, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithTime, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#With Offtime
print("Evaluation for Model With Time 2")
X, y = prepareData(test2)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithTime2, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithTime2, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

print("Evaluation for Model With Time 3")
X, y = prepareData(test3)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithTime3, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithTime3, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

print("Evaluation for Model With Time 4")
X, y = prepareData(test4)
LRy_test_predict, LRy1 = evaluateCVModel(LRfinalModelWithTime4, X, y)
y_test_predict, y1 = evaluateCVModel(finalModelWithTime4, X, y)
predicted.extend(y_test_predict)
ground_truth.extend(y1)
LRpredicted.extend(LRy_test_predict)
LRground_truth.extend(LRy1)

#Metrics On full testset
printmd("**Final Results on complete testset**")

print('Linear Models Best output')
getFinalMetrics(LRpredicted, LRground_truth)

print("Overall Best Model")
getFinalMetrics(predicted, ground_truth)

Evaluation for Model With Time 1


<span style='color:None'>**Model : RidgeCV,  RMSE = 384.68550346136016, R2 =0.5921210321541444**</span>

<span style='color:None'>**Model : RidgeCV,  RMSE = 384.68550346136016, R2 =0.5921210321541444**</span>

Evaluation for Model With Time 2


<span style='color:None'>**Model : LinearRegression,  RMSE = 248.5036411352101, R2 =0.5801771079200388**</span>

...

<span style='color:None'>**Final Results on complete testset**</span>

Linear Models Best output


<span style='color:None'>**RMSE = 379.9509325371697, R2 =0.6891959971474123**</span>

Overall Best Model


<span style='color:None'>**RMSE = 306.5174704336199, R2 =0.7977250792884576**</span>

# Additional Experiments

**Creating a custom Imputer**

In [154]:
class CustomImputerForTemperature(BaseEstimator, TransformerMixin):
    '''
    Class used for imputing missing values in a temperature column using KNNRegressor model, which was found out to be the best*
    
    group_cols : list
        List of columns used for training the regressor model
    target : str
        The name of the column to impute

    Returns
    -------
    X : array-like
        The array with imputed values in the target column
    '''
    def __init__(self, testing=0):       
        self.group_cols = ["Dew point temperature(C)", "Solar Radiation (MJ/m2)", "Snowfall (cm)", "Humidity(%)", "Rainfall(mm)", "Hour_sin", "Hour_cos"]
        self.target = "Temperature(C)"
        self.regressorImputer = KNeighborsRegressor()
        self.testing = testing
    
    def fit(self, X, y=None):
        
        assert pd.isnull(X[self.group_cols]).any(axis=None) == False, 'There are missing values in group_cols'

        for col in self.group_cols:
          if col not in X.columns:
            print("Error: Required Column Not found")
            break

        dfWithTemp = X.loc[X[self.target].notnull()].copy()

        X_train = dfWithTemp[self.group_cols]
        Y_train = dfWithTemp[self.target]

        self.regressorImputer.fit(X_train, Y_train)
        self.impute_regressor_ = self.regressorImputer
        
        return self 
    
    def transform(self, X, y=None):
        
        # make sure that the imputer was fitted
        check_is_fitted(self, 'impute_regressor_')
        
        X = X.copy()
        dfWithTemp = X.loc[X[self.target].notnull()]
        dfPrediction = X.loc[X[self.target].isna()]

        if self.testing == 1:#Required to Check the accuracy of imputer regressor
          X['Temperature(C)'] = self.regressorImputer.predict(X[self.group_cols])
          return X

        if len(dfPrediction) > 0:
          dfPrediction[self.target] = self.regressorImputer.predict(dfPrediction[self.group_cols])
        else:
            return X

        X = pd.concat([dfWithTemp, dfPrediction])
        return X

**Validation and Verification of Custom Imputer**

In [155]:
def customImputerTesting(dataf):
  temperatureImputer = CustomImputerForTemperature()
  data, y = prepareData(dataf)
  A = data['Temperature(C)']
  data = temperatureImputer.fit_transform(data)
  B = data['Temperature(C)']

  #Value to Value comparison
  nanCount = 0
  for i in range(len(A)):
    if np.isnan(A[i]):
      nanCount += 1
      continue
    elif A[i] != B[i]:
      print("Mismatched data, after imputation..")
      break

  uncommon_items_fromA = set(A) - set(B)
  for elem in uncommon_items_fromA:
    if not np.isnan(elem):
      print("Error in Imputation")
      break

  printmd("**No Errors in Custom Imputation, Total : " + str(len(uncommon_items_fromA)) + " Values imputed**")
customImputerTesting(df)

<span style='color:None'>**No Errors in Custom Imputation, Total : 500 Values imputed**</span>

In [156]:
#performance of imputer
X, y = prepareData(df)
dfWithTemp = X.loc[X["Temperature(C)"].notnull()]
dfTempTrain=dfWithTemp.sample(frac=0.8, random_state=42) #random state is a seed value
dfTempTest=dfWithTemp.drop(dfTempTrain.index)

GlobalImputer = CustomImputerForTemperature(True)
GlobalImputer.fit(dfTempTrain)
out = GlobalImputer.transform(dfTempTest)

y_test_predict = out["Temperature(C)"]
y1 = dfTempTest["Temperature(C)"]
print("Evaluation on Test Data (Temperature Imputation )")
getFinalMetrics(y1, y_test_predict, GlobalImputer.regressorImputer)

Evaluation on Test Data (Temperature Imputation )


<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 0.5022672566417514, R2 =0.9982201920189937**</span>

**Stratified Sampling**

In [157]:
stratbikedata = df_new.copy()

stratbikedata["hour_cat"] = pd.cut(stratbikedata["Hour"], bins=[-1, 6, 12, 18, 23], labels=[1,2,3,4]) #when min was given as 0, issue, 0 was classified as Nan

trainDf, testDf = train_test_split(stratbikedata, test_size=0.2, random_state=42, stratify=stratbikedata["hour_cat"])

trainDf.reset_index(drop=True, inplace=True)
testDf.reset_index(drop=True, inplace=True)
X, y = prepareData(trainDf)
soFarBestMSE = sys.maxsize
finalModelAllData = []
LRsoFarBestMSE = sys.maxsize
LRfinalModelAllData = []

for model in models:
  bestModelMSE, imputer, metric = runCV(10, model, X, y)
  if metric < soFarBestMSE:
    finalModelAllData = [bestModelMSE, imputer]
    soFarBestMSE = metric
  if metric < LRsoFarBestMSE:
    if type(model()).__name__ in LinearRegressionModels:
      LRsoFarBestMSE = metric
      LRfinalModelAllData = [bestModelMSE, imputer]

Model --> LinearRegression
RMSE scores --> [439.81112470064807, 418.8871874120276, 460.02970512380494, 429.62504174539623, 427.0845246498865, 431.6634155518132, 404.55270391162185, 432.60638338061835, 425.7176490714148, 450.7261817984651]
R2 scores  --> [0.5476591392710017, 0.5342747364660545, 0.5178854060496993, 0.529654212255495, 0.5475734379963586, 0.5347038652647174, 0.5840627012444755, 0.5219423592305672, 0.5235923788986447, 0.5234914174368948]


<span style='color:None'>**Min RMSE = 404.55270391162185**</span>

<span style='color:None'>**MAX R2 = 0.5840627012444755**</span>

Model --> RidgeCV
RMSE scores --> [439.67543130893915, 418.81337402137837, 460.10341772890126, 429.5335705346812, 427.0197295661073, 431.73710640351896, 404.6442457579303, 432.4174587799555, 425.61246858905594, 450.93301808721844]
R2 scores  --> [0.5479382145081986, 0.5344388557469893, 0.5177308909413101, 0.5298544730153142, 0.5477107072792979, 0.5345449868703487, 0.5838744440617967, 0.5223598155192315, 0.5238277583497923, 0.5230539816500805]


<span style='color:None'>**Min RMSE = 404.6442457579303**</span>

...

<span style='color:None'>**Min RMSE = 406.69967923095254**</span>

<span style='color:None'>**MAX R2 = 0.5796361990057175**</span>

Model --> ElasticNet
RMSE scores --> [482.41046360482, 446.8839192817095, 506.69055453994594, 462.064776353934, 457.559892762768, 467.3079773929718, 446.0345809900278, 461.9678105622849, 456.67491717548285, 489.1186596197332]
R2 scores  --> [0.4557895989756679, 0.4699399194966102, 0.41512353883917164, 0.4559437415645181, 0.4807023905889023, 0.4546876304575036, 0.49439109747344995, 0.45484770108313677, 0.4517865236952421, 0.4388569123149494]


<span style='color:None'>**Min RMSE = 446.0345809900278**</span>

<span style='color:None'>**MAX R2 = 0.49439109747344995**</span>

Testing on Stratified Sampling

In [158]:
X, y = prepareData(testDf)
out = evaluateCVModel(finalModelAllData, X, y)
out = evaluateCVModel(LRfinalModelAllData, X, y)

<span style='color:None'>**Model : KNeighborsRegressor,  RMSE = 300.8225141175441, R2 =0.8051716076015484**</span>

<span style='color:None'>**Model : LinearRegression,  RMSE = 462.80920336140315, R2 =0.5388570778036845**</span>

**Best params for ElasticNet and SGDRegressor Using Grid Search CV**

In [159]:
#scores = GridSCV(df) # Takes time, best values are found out and stored in below list
scores = [{'best_model': ElasticNet(alpha=0.1, l1_ratio=0.99, max_iter=20000), 'best_params': {'alpha': 0.1, 'l1_ratio': 0.99, 'max_iter': 20000}, 'best_score': -203336.15897393713, 'model': 'elasticnet'}, {'best_model': SGDRegressor(alpha=0.1, max_iter=20000, random_state=42), 'best_params': {'alpha': 0.1, 'learning_rate': 'invscaling', 'max_iter': 20000}, 'best_score': -214061.26102864108, 'model': 'SGDRegressor'}]

#for model in scores:
#  models.append(model['best_model'])

**Experiment with XGBoosr**

In [160]:
X, y = prepareData(trainDf)
num_pipeline = Pipeline([('scaler', StandardScaler())])
cat_pipeline = Pipeline([('encoder',OneHotEncoder())])

cat_cols = ['Holiday','Functioning Day']
num_cols = [col for col in list(X) if col not in cat_cols]

full_pipeline = ColumnTransformer([('num', num_pipeline, num_cols),
                                ('cat', cat_pipeline, cat_cols)])

#Data preparation
X_train_prepared = full_pipeline.fit_transform(X)
xgb_r = xg.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
xgb_r.fit(X_train_prepared, y)

X, y = prepareData(testDf)
out = evaluateCVModel([xgb_r, full_pipeline], X, y)

[13:13:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


<span style='color:None'>**Model : XGBRegressor,  RMSE = 223.69385632142476, R2 =0.8922692491047886**</span>

In [161]:
#End